In [1]:
import os
import cv2
from xml.etree import ElementTree as ET
import matplotlib.pyplot as plt
import json

In [2]:
img_path = '/mnt/6B133E147DED759E/Archive2017.8.16'
#xml_files = []
#img_files = []
anno_dict = {}
for root, dirs, files in os.walk(img_path):
    for name in files:
        if name.endswith(".xml"):
            anno_dict[os.path.join(root, name)] = os.path.join(root, name[:-4]+'.jpg')
            

In [3]:
dst_path = '/mnt/6B133E147DED759E/Archive2017.8.16_res'
xpad = 100
ypad = 50
scale = 0.4
new_annos = []

In [4]:
k = 0
for annoiter in anno_dict.iteritems():
    img = cv2.imread(annoiter[1])
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    anno = ET.parse(annoiter[0])
    obj_node=anno.getiterator("object")
    rects = []
    for obj in obj_node:
        bndbox = obj.find('bndbox')
        name = obj.find('name')
        if name.text =='obj' or name.text == 'x':
            c = 0
        else:
            c = 1
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)
        
        xmin_inner = max(0,int(xmin - xpad))
        ymin_inner = max(0,int(ymin - ypad))
        xmax_inner = min(img.shape[1]-1, int(xmax + xpad))
        ymax_inner = min(img.shape[0]-1, int(ymax + ypad))
        
        roi = img[ymin_inner:ymax_inner+1, xmin_inner:xmax_inner+1,:]
        roi = cv2.resize(roi, (0,0), fx=scale, fy=scale)
        
        xmin = (xmin -xmin_inner)*scale
        xmax = (xmax -xmin_inner)*scale
        ymin = (ymin- ymin_inner)*scale
        ymax = (ymax -ymin_inner)*scale
        
        xmin = max(0,int(xmin))
        ymin = max(0,int(ymin))
        xmax = min(roi.shape[1]-1, int(xmax))
        ymax = min(roi.shape[0]-1, int(ymax))
        
        cv2.imwrite(dst_path+'/'+str(k) + '.jpg', roi)
        
        new_annos.append({'filename': str(k) + '.jpg', 'type': c, 'xmin':xmin, 'ymin':ymin, 'xmax':xmax, 'ymax':ymax})
        k = k + 1
#         rects.append(((xmin, ymin), (xmax, ymax), c))
#     for r in rects:
#         if r[2]:
#             cv2.rectangle(img, r[0], r[1], (255,0,0),16)
#         else:
#             cv2.rectangle(img, r[0], r[1], (0,255,0),16)
#     plt.imshow(img)
#     plt.show()

In [5]:
with open(dst_path + '/data.json', 'w') as json_file:
    json_file.write(json.dumps(new_annos))